## First let's set up our bike crashes file

In [1]:
import pandas as pd

file = 'janfebnyccrashes.xlsx' #excel file with january/february accidents
df = pd.read_excel(file)

df.info() #2000-2500 accidents total including car to car collisions.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2510 entries, 0 to 2509
Data columns (total 11 columns):
CRASH DATE                       2510 non-null datetime64[ns]
CRASH TIME                       2510 non-null object
LATITUDE                         2413 non-null float64
LONGITUDE                        2413 non-null float64
LOCATION                         2413 non-null object
NUMBER OF CYCLIST INJURED        2510 non-null int64
NUMBER OF CYCLIST KILLED         2510 non-null int64
CONTRIBUTING FACTOR VEHICLE 1    2501 non-null object
CONTRIBUTING FACTOR VEHICLE 2    2104 non-null object
VEHICLE TYPE CODE 1              2489 non-null object
VEHICLE TYPE CODE 2              2008 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 215.8+ KB


In [2]:
## Let's get only the rows with cyclists involved

#all accidents involving bikes, anything above 0 
df = df[(df['NUMBER OF CYCLIST INJURED'] > 0) | (df['NUMBER OF CYCLIST KILLED'] > 0)] 

df = df.reset_index(drop=True) # drop index and create dataframe

In [3]:
df.info() # 72 data points, and some missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 11 columns):
CRASH DATE                       72 non-null datetime64[ns]
CRASH TIME                       72 non-null object
LATITUDE                         70 non-null float64
LONGITUDE                        70 non-null float64
LOCATION                         70 non-null object
NUMBER OF CYCLIST INJURED        72 non-null int64
NUMBER OF CYCLIST KILLED         72 non-null int64
CONTRIBUTING FACTOR VEHICLE 1    72 non-null object
CONTRIBUTING FACTOR VEHICLE 2    69 non-null object
VEHICLE TYPE CODE 1              72 non-null object
VEHICLE TYPE CODE 2              66 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 6.3+ KB


In [6]:
df = df[df['VEHICLE TYPE CODE 2'].notna()]
df = df[df['LOCATION'].notna()] # if no location, then drop it

In [8]:
# clean up column titles for easier accessiblity
df.columns = [x.lower() for x in df.columns] # lower case columns
df.columns = [x.replace(' ', '_').lower() for x in df.columns] #replace spaces with '_' for accessibility
list(df.columns)

['crash_date',
 'crash_time',
 'latitude',
 'longitude',
 'location',
 'number_of_cyclist_injured',
 'number_of_cyclist_killed',
 'contributing_factor_vehicle_1',
 'contributing_factor_vehicle_2',
 'vehicle_type_code_1',
 'vehicle_type_code_2']

In [9]:
df.shape # 65 rows, 11 features

(65, 11)

## Now subset only crash related details excluding location

In [10]:
# Let's get the accidents in each tooltip
## We'll remove the coordinates (they'll be used in a future analysis)
accidents_info_df = df[['crash_date','crash_time','number_of_cyclist_injured','number_of_cyclist_killed',
                        'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'vehicle_type_code_1',
                       'vehicle_type_code_2']]


In [12]:
accidents_info_df.shape # removed 3 columns related to location coordinates

(65, 8)

## Querying crash details

### How did drivers cause the accidents?

In [51]:
# query reasons how bikers cause the accidents
accidents_info_df[accidents_info_df['vehicle_type_code_2'] == 'Bike']['contributing_factor_vehicle_1'].value_counts()

Driver Inattention/Distraction                           19
Unspecified                                              11
Passing or Lane Usage Improper                            6
Passenger Distraction                                     4
Failure to Yield Right-of-Way                             3
Pedestrian/Bicyclist/Other Pedestrian Error/Confusion     3
Following Too Closely                                     2
Turning Improperly                                        2
Backing Unsafely                                          2
Traffic Control Disregarded                               1
Passing Too Closely                                       1
Unsafe Lane Changing                                      1
Driver Inexperience                                       1
Name: contributing_factor_vehicle_1, dtype: int64

### How did bikers cause the accidents?

In [16]:
accidents_info_df[accidents_info_df['vehicle_type_code_1'] == 'Bike']['contributing_factor_vehicle_1'].value_counts()

Driver Inattention/Distraction    6
Aggressive Driving/Road Rage      1
Passing Too Closely               1
Unspecified                       1
Name: contributing_factor_vehicle_1, dtype: int64

### How did both parties contribute to the accident?

In [43]:
both_parties_df = accidents_info_df[(accidents_info_df['contributing_factor_vehicle_1'] != 'Unspecified') & 
                  (accidents_info_df['contributing_factor_vehicle_2'] != 'Unspecified')][[
    'contributing_factor_vehicle_1','contributing_factor_vehicle_2', 'vehicle_type_code_1', 'vehicle_type_code_2']].reset_index(drop=True)
both_parties_df

,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code_1,vehicle_type_code_2
0,Driver Inattention/Distraction,Driver Inattention/Distraction,Bike,Sedan
1,Turning Improperly,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Station Wagon/Sport Utility Vehicle,Bike
2,Backing Unsafely,Passing Too Closely,Sedan,Bike
3,Passenger Distraction,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Taxi,Bike
4,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Taxi,Bike
5,Driver Inattention/Distraction,Driver Inattention/Distraction,Sedan,Bike
6,Backing Unsafely,Following Too Closely,Station Wagon/Sport Utility Vehicle,Bike
7,Driver Inattention/Distraction,Failure to Yield Right-of-Way,Taxi,Bike
8,Driver Inattention/Distraction,Driver Inattention/Distraction,Bike,Sedan
9,Driver Inattention/Distraction,Driver Inattention/Distraction,Taxi,Bike


### How did taxi drivers cause the accidents?

In [14]:
#query reasons how taxis cause the accdients
accidents_info_df[(accidents_info_df['vehicle_type_code_1'] == 'Taxi')]['contributing_factor_vehicle_1'].value_counts()

Driver Inattention/Distraction                           6
Passing or Lane Usage Improper                           4
Unspecified                                              3
Following Too Closely                                    2
Passenger Distraction                                    2
Pedestrian/Bicyclist/Other Pedestrian Error/Confusion    2
Traffic Control Disregarded                              1
Name: contributing_factor_vehicle_1, dtype: int64

### Let's compare this to Sedans

In [47]:
#query reasons how sedans cause the accdients
accidents_info_df[(accidents_info_df['vehicle_type_code_1'] == 'Station Wagon/Sport Utility Vehicle')]['contributing_factor_vehicle_1'].value_counts()

Unspecified                                              6
Driver Inattention/Distraction                           4
Failure to Yield Right-of-Way                            1
Turning Improperly                                       1
Passing or Lane Usage Improper                           1
Backing Unsafely                                         1
Passenger Distraction                                    1
Pedestrian/Bicyclist/Other Pedestrian Error/Confusion    1
Name: contributing_factor_vehicle_1, dtype: int64

### Were there any bike to bike collisions?

In [15]:
# Bike to Bike Collisions?
df[(df['vehicle_type_code_1'] == 'Bike') & (df['vehicle_type_code_2'] == 'Bike')] # one incident

,crash_date,crash_time,latitude,longitude,location,number_of_cyclist_injured,number_of_cyclist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code_1,vehicle_type_code_2
34,2020-01-22,13:49:00,40.74881,-74.000946,POINT (-74.000946 40.74881),1,0,Passing Too Closely,Unspecified,Bike,Bike
